In [222]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Importing the dataset
Firstly, we import the dataset we will use.
Feel free to change the dataset to another tournament

In [223]:
# You can change the file name to any tournament you want to analyze
file_name = "LFL_2025_Flash_In_Swiss.csv"
dataset = pd.read_csv("data/" + file_name)

In [224]:
# Compute this code if you want to merge 2 datasets
file_name2 = "LFL_2025_Flash_In_Playoffs.csv"
dataset2 = pd.read_csv("data/" + file_name2)
dataset = pd.concat([dataset, dataset2])

file_name3 = "LFL_2025_Flash_In_Groups.csv"
dataset3 = pd.read_csv("data/" + file_name3)
dataset = pd.concat([dataset, dataset3])


In [225]:
# View the first 10 rows of the dataset
dataset.head(10)

,Unnamed: 0,gameId,patch,participantId,blueTeam,redTeam,role,PlayerScore0,PlayerScore1,PlayerScore10,...,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,0,96141,15.2,1,Ici Japon Corp. Esport,Vitality.Bee,TOP,0.0,0.0,0.0,...,1,2,3,0,0,39,4,9,9,True
1,1,96141,15.2,2,Ici Japon Corp. Esport,Vitality.Bee,JUNGLE,0.0,0.0,0.0,...,2,4,3,0,0,47,12,12,14,True
2,2,96141,15.2,3,Ici Japon Corp. Esport,Vitality.Bee,MID,0.0,0.0,0.0,...,1,4,3,0,0,32,9,4,19,True
3,3,96141,15.2,4,Ici Japon Corp. Esport,Vitality.Bee,ADC,0.0,0.0,0.0,...,3,6,3,0,0,34,3,9,15,True
4,4,96141,15.2,5,Ici Japon Corp. Esport,Vitality.Bee,SUPPORT,0.0,0.0,0.0,...,1,2,3,0,0,122,23,16,68,True
5,5,96141,15.2,6,Ici Japon Corp. Esport,Vitality.Bee,TOP,0.0,0.0,0.0,...,0,1,9,0,0,32,9,4,15,False
6,6,96141,15.2,7,Ici Japon Corp. Esport,Vitality.Bee,JUNGLE,0.0,0.0,0.0,...,1,1,9,0,0,56,7,14,7,False
7,7,96141,15.2,8,Ici Japon Corp. Esport,Vitality.Bee,MID,0.0,0.0,0.0,...,0,0,9,0,0,53,8,11,18,False
8,8,96141,15.2,9,Ici Japon Corp. Esport,Vitality.Bee,ADC,0.0,0.0,0.0,...,2,2,9,0,0,31,4,6,13,False
9,9,96141,15.2,10,Ici Japon Corp. Esport,Vitality.Bee,SUPPORT,0.0,0.0,0.0,...,0,0,9,0,0,123,16,15,57,False


In [226]:
# Showing all the possible attributes

dataset.columns

Index(['Unnamed: 0', 'gameId', 'patch', 'participantId', 'blueTeam', 'redTeam',
       'role', 'PlayerScore0', 'PlayerScore1', 'PlayerScore10',
       ...
       'turretKills', 'turretTakedowns', 'turretsLost', 'unrealKills',
       'visionClearedPings', 'visionScore', 'visionWardsBoughtInGame',
       'wardsKilled', 'wardsPlaced', 'win'],
      dtype='object', length=149)

# Selecting good attributes
The attributes selected are the ones that are useful for a possible player rating.
The `champExperience` is only available in the V5 of stats API and is not available in the V4.
This is why we make a verification in the dataset if it contains the `champExperience` attribute. Otherwise, we will use the `champLevel` attribute.
```python
if 'champExperience' in dataset.columns:
    useful_columns.append('champExperience')
else:
    useful_columns.append('champLevel')
```

In [227]:
# List of possible useful columns
# This is a prefilter of the columns
useful_columns = ['gameId', 
                  'patch', 
                  'role', 
                  'riotIdGameName',
                  'blueTeam',
                  'redTeam',
                  'teamId',
                  'championName',
                  'kills',
                  'deaths',
                  'assists',
                  'goldEarned',
                  'totalDamageDealtToChampions',
                  'totalMinionsKilled',
                  'totalHeal',
                  'timeCCingOthers',
                  'totalDamageTaken',
                  'damageSelfMitigated',
                  'wardsKilled',
                  'wardsPlaced',
                  'win',
                  ]
# If 'champExperience' is in the dataset, then we can use it
# otherwise, we can use 'champLevel'
if 'champExperience' in dataset.columns:
    useful_columns.append('champExperience')
else:
    useful_columns.append('champLevel')

final_dataset = dataset[useful_columns].copy()

# Let's rename the champLevel to champExperience if it is not in the dataset
if 'champExperience' not in dataset.columns:
    final_dataset.rename(columns={'champLevel': 'champExperience'}, inplace=True)

final_dataset.head(10)

,gameId,patch,role,riotIdGameName,blueTeam,redTeam,teamId,championName,kills,deaths,...,totalDamageDealtToChampions,totalMinionsKilled,totalHeal,timeCCingOthers,totalDamageTaken,damageSelfMitigated,wardsKilled,wardsPlaced,win,champExperience
0,96141,15.2,TOP,IJC Howling,Ici Japon Corp. Esport,Vitality.Bee,100,Ambessa,6,0,...,36317,250,10194,17,32950,41443,9,9,True,18697
1,96141,15.2,JUNGLE,IJC Theocacs,Ici Japon Corp. Esport,Vitality.Bee,100,XinZhao,3,5,...,24115,35,31830,41,53874,47786,12,14,True,16699
2,96141,15.2,MID,IJC Sertuss,Ici Japon Corp. Esport,Vitality.Bee,100,Viktor,6,2,...,38156,229,2002,35,16122,19447,4,19,True,17379
3,96141,15.2,ADC,IJC Hans SamD,Ici Japon Corp. Esport,Vitality.Bee,100,MissFortune,11,3,...,45856,314,10622,14,20085,21840,9,15,True,18805
4,96141,15.2,SUPPORT,IJC Prime,Ici Japon Corp. Esport,Vitality.Bee,100,Rell,0,10,...,7483,31,576,33,24584,25103,16,68,True,9510
5,96141,15.2,TOP,VITB Vertigo,Ici Japon Corp. Esport,Vitality.Bee,200,Renekton,3,3,...,15002,250,11673,25,45225,44284,4,15,False,15554
6,96141,15.2,JUNGLE,VITB Lurox,Ici Japon Corp. Esport,Vitality.Bee,200,Zyra,2,5,...,25122,14,10840,60,29326,16607,14,7,False,13479
7,96141,15.2,MID,VITB Vetheo,Ici Japon Corp. Esport,Vitality.Bee,200,LeeSin,4,7,...,25768,229,14044,30,49989,48462,11,18,False,14770
8,96141,15.2,ADC,VITB Jopa,Ici Japon Corp. Esport,Vitality.Bee,200,Corki,8,3,...,38265,288,4485,1,22649,14910,6,13,False,15834
9,96141,15.2,SUPPORT,VITB Fleshy,Ici Japon Corp. Esport,Vitality.Bee,200,Braum,3,8,...,8469,36,3855,66,29004,37013,15,57,False,8497


In [228]:
# Rename riotIdGameName to summonerName
final_dataset.rename(columns={'riotIdGameName': 'summonerName'}, inplace=True)

In [229]:
summoner_names_with_2_spaces = final_dataset[final_dataset['summonerName'].str.count(' ') == 2]['summonerName']
print(summoner_names_with_2_spaces)


3      IJC Hans SamD
18     IJC Hans SamD
148    IJC Hans SamD
153    IJC Hans SamD
168    IJC Hans SamD
3      IJC Hans SamD
13     IJC Hans SamD
23     IJC Hans SamD
83     IJC Hans SamD
98     IJC Hans SamD
108    IJC Hans SamD
113    IJC Hans SamD
128    IJC Hans SamD
133    IJC Hans SamD
143    IJC Hans SamD
158    IJC Hans SamD
18     IJC Hans SamD
43     IJC Hans SamD
83     IJC Hans SamD
168    IJC Hans SamD
Name: summonerName, dtype: object


# Cleaning data
Since some players might have renamed while the tournament (like Targamas who used XL Targamas instead of KC Targamas when he was in KC) we will replace the names with the most used one in the tournament.

In [230]:
from collections import defaultdict

def fix_tags(dataset):
    # Create a dictionary to store player information
    player_info = defaultdict(list)

    # Traverse the dataset to collect player information
    for entry in dataset:
        # Convert entry to string and split into parts
        entry = str(entry)
        parts = entry.split(' ', 1)

        if len(parts) == 2:
            tag, summoner_name = parts
            player_info[summoner_name].append(tag)
        else:
            # If there is only one part, consider it as the summoner name
            summoner_name = parts[0]

    # Identify players who used tags from old teams
    players_to_fix = {name: tags for name,
                      tags in player_info.items() if len(tags) > 1}

    # Replace tags from old teams with the most common tags
    fixed_dataset = []
    for entry in dataset:
        entry = str(entry)
        parts = entry.split(' ', 1)

        if len(parts) == 2:
            tag, summoner_name = parts
            if summoner_name in players_to_fix:
                fixed_tag = max(
                    player_info[summoner_name], key=player_info[summoner_name].count)
                entry = f"{fixed_tag} {summoner_name}"

        fixed_dataset.append(entry)

    return fixed_dataset

# Convert the column to string to avoid issues with non-string values
final_dataset["summonerName"] = final_dataset["summonerName"].astype(str)

# Apply the fix_tags function
fixed_summoner_names = fix_tags(final_dataset["summonerName"])

# Assign the fixed dataset to the "summonerName" column
final_dataset["summonerName"] = fixed_summoner_names

# Print the players that are present less than 10 times
player_counts = final_dataset["summonerName"].value_counts()
print(player_counts)

summonerName
SLY Kryze        21
SLY Mersa        21
SLY TakeSet      21
SLY Decay        21
SLY Markoon      21
IJC Theocacs     20
IJC Howling      20
IJC Hans SamD    20
IJC Sertuss      20
IJC Prime        20
KCB Maynter      18
KCB Piero        18
KCB Boukada      18
KCB SlowQ        18
KCB 3XA          18
GL Jezu          16
GL SeRiN         16
GL Xerxe         16
GL Ragner        16
GL Zoelys        16
BKR Doss         12
BKR Keduii       12
BKR OMON         12
BKR Rhilech      12
BKR Szygenda     12
M8 Empyros       11
M8 Eika          11
M8 Comp          11
M8 Erdote        11
M8 Zicssi        11
JL Twiizt        10
JL Deadly        10
JL Diplex        10
JL RAMES         10
JL Leny          10
VITB Vetheo       8
VITB Jopa         8
VITB Vertigo      8
VITB Lurox        8
VITB Fleshy       8
GW Zest           6
GW Stormax        6
GW Backlund       6
GW Matias         6
GW HONOR          6
BDSA Hiro1        4
BDSA Shlatan      4
BDSA Toffe        4
BDSA Piwek        4
BDSA My

# Analysis of the data

In [231]:
# Possibility to make a analysis of the data

# profile = ProfileReport(final_dataset, title="Profiling Report")
# profile.to_notebook_iframe()

# Derived attributes
In his [analysis](https://medium.com/@benjamin.castet/rate-a-single-performance-league-of-legends-and-other-competitive-games-1e45fe7b9fdf) of the stats, Xenesis used some derived attributes. I'll try to explain them here. 

## Champ-Role-Patch
Firstly, we have the 'Champ-Role-Patch' attribute. It is the concatenation of the champion, the role and the patch. It is used to group the stats by champion, role and patch. As Xenesis writed explicitly, this is useful to make a difference between the stats of a champion in a different role. For example, Mid Seraphine and Support Seraphine are not the same champion since Mid Seraphine is played more as a poke champion and Support Seraphine is played more as an enchanter. 

Also we add the patch in this attribute because the build or the champion can change through the patches. For example, Aatrox can be built as a bruiser or as a tank in patch 11.1 and the next patch nerfs the tank build but the champion is still playable with lethality. 


In [232]:
# Make column for Champion-Role-Patch
final_dataset['champ-role-patch'] = final_dataset['championName'] + '-' + final_dataset['role'] + '-' + final_dataset['patch'].astype(str)

## Players stats against whole opponent team
Something good that I saw from @lol_Xenesis on Twitter is to compare the values of every players stats 
against the whole opponent team.
It will basically give you something like :
- playerWardsKilled / wholeOpponentTeamWardsKilled

This is kinda smart because it doesn't require to compare with the main matchup since you can have weak side and strong sides. 
By doing this we will see if each players had a advantage over the opponents.

For example, imagine a composition with a advantage in the matchup on top. It will be normal to see the top laner having a better stats than the opponent. But obviously, the opponent will find a draft with some advantages on the other lanes (with a strong bot lane for example). This is why we need to compare the stats of the top laner with the whole opponent team.

In [233]:
# Let's try to make new features

# If we make a basic (K+A)/D it will tend to inf since it's possible to have division by 0 (deaths)
# final_dataset["kda"] =  (final_dataset["kills"] + final_dataset["assists"]) / final_dataset["deaths"] 
# DON'T DO THIS 


final_dataset["k+a"] = final_dataset["kills"] + final_dataset["assists"]

# ---------------------------
# First rename the columns
# ---------------------------

# Rename the column "totalGold" to "gold"
final_dataset.rename(columns={"goldEarned": "gold"}, inplace=True)

# Rename totalHeal to heal
final_dataset.rename(columns={"totalHeal": "heal"}, inplace=True)

# Rename totalDamageDealtToChampions to damage
final_dataset.rename(columns={"totalDamageDealtToChampions": "damage"}, inplace=True)

# Rename totalDamageTaken to damageTaken
final_dataset.rename(columns={"totalDamageTaken": "damageTaken"}, inplace=True)

# Rename totalMinionsKilled to cs
final_dataset.rename(columns={"totalMinionsKilled": "cs"}, inplace=True)

# Rename timeCCingOthers to cc
final_dataset.rename(columns={"timeCCingOthers": "cc"}, inplace=True)

In [234]:
# Deaths
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['deaths'].transform('sum')
# If the total is equal to 0, then we set it to 0.01 to avoid division by 0
final_dataset['total'] = final_dataset['total'].apply(lambda x: 0.01 if x == 0 else x)

final_dataset['n_deaths'] = final_dataset.apply(lambda row: row['deaths'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)
contains_infinite = np.isinf(final_dataset['n_deaths']).any()
print(contains_infinite)

False


In [235]:
# k+a
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['k+a'].transform('sum')
# Add k+a of the player game in total column
# Otherwise it can have infinite values
final_dataset['total'] = final_dataset.apply(lambda row: row['total'] + final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['k+a'].iloc[0], axis=1)

final_dataset['n_k+a'] = final_dataset.apply(lambda row: row['k+a'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)

contains_infinite = np.isinf(final_dataset['n_k+a']).any()
print(contains_infinite)
#final_dataset[["k+a", "total","n_k+a"]].head(10)

False


In [236]:
# damage
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['damage'].transform('sum')
final_dataset['n_damage'] = final_dataset.apply(lambda row: row['damage'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)

In [237]:
# gold
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['gold'].transform('sum')
final_dataset['n_gold'] = final_dataset.apply(lambda row: row['damage'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)

In [238]:
# cs
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['cs'].transform('sum')
final_dataset['n_cs'] = final_dataset.apply(lambda row: row['cs'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)

In [239]:
# champLevel
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['champExperience'].transform('sum')
final_dataset['n_champExperience'] = final_dataset.apply(lambda row: row['champExperience'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)


In [240]:
# wardsPlaced
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['wardsPlaced'].transform('sum')
final_dataset['n_wardsPlaced'] = final_dataset.apply(lambda row: row['wardsPlaced'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)

In [241]:
# wardsKilled
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['wardsKilled'].transform('sum')
final_dataset['n_wardsKilled'] = final_dataset.apply(lambda row: row['wardsKilled'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)

In [242]:
# heal
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['heal'].transform('sum')
final_dataset['n_heal'] = final_dataset.apply(lambda row: row['heal'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)

In [243]:
# damageSelfMitigated
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['damageSelfMitigated'].transform('sum')
final_dataset['n_damageSelfMitigated'] = final_dataset.apply(lambda row: row['damageSelfMitigated'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)

In [244]:
# cc
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['cc'].transform('sum')
final_dataset['n_cc'] = final_dataset.apply(lambda row: row['cc'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)

In [245]:
# damageTaken
final_dataset['total'] = final_dataset.groupby(['gameId', 'teamId'])['damageTaken'].transform('sum')
final_dataset['n_damageTaken'] = final_dataset.apply(lambda row: row['damageTaken'] / final_dataset[(final_dataset['gameId'] == row['gameId']) & (final_dataset['teamId'] != row['teamId'])]['total'].iloc[0], axis=1)

final_dataset = final_dataset.drop(['total'], axis=1)

In [246]:
# profile = ProfileReport(final_dataset, title="Profiling Report")
# profile.to_file("profiling_report.html")
# # Now let's show the report with the new attributes

# Checks
Here we are checking the number of games for each champion, role and patch

In [247]:

champ_role_patch_counts = final_dataset['champ-role-patch'].value_counts()

print(champ_role_patch_counts)

champ-role-patch
KSante-TOP-15.1           12
Kalista-ADC-15.1          10
Rell-SUPPORT-15.2         10
Rell-SUPPORT-15.1         10
Ezreal-ADC-15.2           10
                          ..
Ahri-MID-15.3              1
Swain-TOP-15.3             1
Seraphine-SUPPORT-15.3     1
Varus-ADC-15.3             1
Viego-JUNGLE-15.1          1
Name: count, Length: 205, dtype: int64


# Correlation coefficients
Now we want to get the correlation coefficients for each champ-role-patch possible value. 

As said before, if a champion is played as support in a game, its stats won't be the same as if it was played as mid. Also the goal of the champion will probably not be the same. So we want to get the correlation coefficients for each champ-role-patch possible value.

In [248]:

# Load or define your dataset as 'final_dataset'
# Define the column on which you want to apply the filter
filter_column = 'champ-role-patch'

# Define the numeric columns for which you want to calculate correlations
numeric_columns = final_dataset.filter(regex='^n_').columns.tolist() + ['win']

# Create an empty dictionary to store correlation coefficients for each group
correlation_coefficients = {}

# Group by the filter_column and calculate correlation matrices for each group
for group_name, group_data in final_dataset.groupby(filter_column):
    correlation_matrix = group_data[numeric_columns].corr()
    win_correlation = correlation_matrix['win'].drop('win')  # Get correlation coefficients for 'win'
    # Remove the win coefficient from the correlation matrix

    correlation_coefficients[group_name] = win_correlation

# Print or display the correlation coefficients for each group
for group_name, win_correlation in correlation_coefficients.items():
    print(f"Correlation coefficients for 'win' {group_name}:\n{win_correlation}")
    print("\n")


Correlation coefficients for 'win' Aatrox-TOP-15.3:
n_deaths                NaN
n_k+a                   NaN
n_damage                NaN
n_gold                  NaN
n_cs                    NaN
n_champExperience       NaN
n_wardsPlaced           NaN
n_wardsKilled           NaN
n_heal                  NaN
n_damageSelfMitigated   NaN
n_cc                    NaN
n_damageTaken           NaN
Name: win, dtype: float64


Correlation coefficients for 'win' Ahri-MID-15.2:
n_deaths                -0.998111
n_k+a                    0.968229
n_damage                 0.999930
n_gold                   0.278978
n_cs                     0.187683
n_champExperience        0.955927
n_wardsPlaced            0.950711
n_wardsKilled           -0.053387
n_heal                   0.562938
n_damageSelfMitigated    0.672217
n_cc                    -0.561815
n_damageTaken           -0.001605
Name: win, dtype: float64


Correlation coefficients for 'win' Ahri-MID-15.3:
n_deaths                NaN
n_k+a               

# Standardization
We need to standardize the data before making any calculations. By doing this, we ensure that the data is comparable and all in the same scale (without coefficients).
 

In [249]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load or define your dataset as 'final_dataset'
# For example:
# final_dataset = pd.read_csv('your_dataset.csv')

# Select numeric columns that start with 'n_'
numeric_columns = final_dataset.filter(regex='^n_').columns.tolist()

# Create a StandardScaler instance
scaler = StandardScaler()

# Apply standardization to the selected numeric columns
final_dataset[numeric_columns] = scaler.fit_transform(final_dataset[numeric_columns])

# Now the columns starting with 'n_' are standardized

In [250]:
import pandas as pd

# Load or define your dataset as 'final_dataset'
# Define the column on which you want to apply the filter
filter_column = 'champ-role-patch'

# Create a copy of the dataset to store the ratings
ratings_dataset = final_dataset.copy()

# Apply ratings to the numeric columns based on correlation coefficients
for group_name, win_correlation in correlation_coefficients.items():
    group_indices = final_dataset[filter_column] == group_name
    ratings = final_dataset.loc[group_indices, numeric_columns] * win_correlation.values
    ratings_dataset.loc[group_indices, 'rating'] = ratings.sum(axis=1)



# Ratings
So basically, we have 2 types of ratings:
- **Positive** ratings 
- **Negative** ratings

When the rating is positive it means that the player is better than the average player of his role, champion and patch
When the rating is negative it means that the player is worse than the average player of his role, champion and patch

In [251]:
ratings_dataset.tail(10)

,gameId,patch,role,summonerName,blueTeam,redTeam,teamId,championName,kills,deaths,...,n_gold,n_cs,n_champExperience,n_wardsPlaced,n_wardsKilled,n_heal,n_damageSelfMitigated,n_cc,n_damageTaken,rating
220,95981,15.1,TOP,JL Leny,Joblife,Karmine Corp Blue,100,KSante,0,3,...,-0.593973,0.672410,0.059501,-0.537683,-1.284445,-0.508166,3.815867,-0.277110,1.691311,-1.886558
221,95981,15.1,JUNGLE,JL RAMES,Joblife,Karmine Corp Blue,100,Viego,3,4,...,-1.099544,-1.332501,-0.693796,-0.537683,-0.378254,0.741519,0.277727,-0.977866,2.019806,0.000000
222,95981,15.1,MID,JL Diplex,Joblife,Karmine Corp Blue,100,Aurora,4,5,...,0.688049,0.427268,-0.351114,-0.766775,-0.831349,-0.669252,-0.851148,-0.708344,-0.203881,-1.283024
223,95981,15.1,ADC,JL Deadly,Joblife,Karmine Corp Blue,100,Corki,1,2,...,0.084750,1.092653,-0.051885,-0.537683,0.754484,-0.736172,-0.959988,-1.085674,-0.760067,0.656500
224,95981,15.1,SUPPORT,JL Twiizt,Joblife,Karmine Corp Blue,100,Nautilus,0,6,...,-1.485185,-1.122379,-2.005947,1.237779,0.074841,-0.874219,-0.426098,-0.384918,-0.784387,-4.750788
225,95981,15.1,TOP,KCB Maynter,Joblife,Karmine Corp Blue,200,Rumble,3,3,...,0.782871,0.496094,0.733130,-0.467765,-0.926353,-0.560464,-0.644440,1.146194,-0.738121,2.257398
226,95981,15.1,JUNGLE,KCB Boukada,Joblife,Karmine Corp Blue,200,Sejuani,2,1,...,-0.217149,-1.229938,0.221167,-0.467765,0.696020,1.150204,0.344142,2.316032,0.156342,2.088365
227,95981,15.1,MID,KCB SlowQ,Joblife,Karmine Corp Blue,200,Yone,1,2,...,0.236537,1.004819,1.748995,-0.182144,-1.196749,0.973260,-0.096095,0.795242,0.333477,6.416790
228,95981,15.1,ADC,KCB 3XA,Joblife,Karmine Corp Blue,200,Kaisa,11,0,...,1.965924,1.095663,1.042951,-0.110739,2.047998,-0.196280,-1.085801,-0.608564,-1.146005,7.522821
229,95981,15.1,SUPPORT,KCB Piero,Joblife,Karmine Corp Blue,200,Alistar,3,2,...,-1.138223,-1.111841,-0.684439,2.531256,0.696020,0.680360,-0.348282,2.316032,-0.591397,3.721722


In [252]:
# Now we will make an average of the ratings for each summonerName

# Group by 'summonerName' and 'champ-role-patch', calculate the average rating and count for each group
grouped_data = ratings_dataset.groupby(['summonerName', 'role'])['rating'].agg(['mean', 'count'])

# Create a new DataFrame to store the average ratings, counts, summoner names, and champ-role-patch
average_ratings_df = pd.DataFrame({
    'summonerName': grouped_data.index.get_level_values('summonerName'),
    'role': grouped_data.index.get_level_values('role'),
    'average_rating': grouped_data['mean'],
    'count': grouped_data['count']
}).reset_index(drop=True)  # Reset index and drop the old index levels

sorted_average_ratings_df = average_ratings_df.sort_values(by='average_rating', ascending=False)
print(sorted_average_ratings_df.head(10))

     summonerName role  average_rating  count
34      KCB SlowQ  MID        3.506830     18
10        GL Jezu  ADC        2.943578     16
12       GL SeRiN  MID        2.758774     16
20  IJC Hans SamD  ADC        2.640626     20
30        KCB 3XA  ADC        2.591184     18
6      BKR Keduii  ADC        2.418319     12
32    KCB Maynter  TOP        1.696517     18
11      GL Ragner  TOP        1.520871     16
44    SLY TakeSet  ADC        1.434728     21
9    BKR Szygenda  TOP        1.143319     12


In [253]:
# Group by 'role' and display the top 5 players for each role
for role, group_data in sorted_average_ratings_df.groupby('role'):
    top_players = group_data.head(5)
    print(f"Top 5 players for role '{role}':\n{top_players}\n")

Top 5 players for role 'ADC':
     summonerName role  average_rating  count
10        GL Jezu  ADC        2.943578     16
20  IJC Hans SamD  ADC        2.640626     20
30        KCB 3XA  ADC        2.591184     18
6      BKR Keduii  ADC        2.418319     12
44    SLY TakeSet  ADC        1.434728     21

Top 5 players for role 'JUNGLE':
   summonerName    role  average_rating  count
13     GL Xerxe  JUNGLE        1.084402     16
8   BKR Rhilech  JUNGLE        0.329233     12
31  KCB Boukada  JUNGLE        0.098797     18
42  SLY Markoon  JUNGLE       -0.348723     21
47   VITB Lurox  JUNGLE       -0.592021      8

Top 5 players for role 'MID':
   summonerName role  average_rating  count
34    KCB SlowQ  MID        3.506830     18
12     GL SeRiN  MID        2.758774     16
7      BKR OMON  MID        1.135833     12
15  GW Backlund  MID        0.580094      6
23  IJC Sertuss  MID        0.425366     20

Top 5 players for role 'SUPPORT':
   summonerName     role  average_rating  count


In [257]:
# Define the role you want to display
target_role = 'TOP'  # Replace with the desired role

# Filter the DataFrame for the target role and display the top 5 players
top_players_target_role = sorted_average_ratings_df[sorted_average_ratings_df['role'] == target_role]
print(f"Players for role '{target_role}':\n{top_players_target_role}")

Players for role 'TOP':
    summonerName role  average_rating  count
32   KCB Maynter  TOP        1.696517     18
11     GL Ragner  TOP        1.520871     16
9   BKR Szygenda  TOP        1.143319     12
37    M8 Empyros  TOP        0.192133     11
21   IJC Howling  TOP        0.181720     20
41     SLY Kryze  TOP       -0.044538     21
19       GW Zest  TOP       -0.259051      6
48  VITB Vertigo  TOP       -0.539168      8
27       JL Leny  TOP       -1.832291     10
0     BDSA Hiro1  TOP       -2.262686      4


In [255]:
# See all the ratings of the summonerName

ratings_dataset[ratings_dataset['summonerName'] == 'BDS Adam']


,gameId,patch,role,summonerName,blueTeam,redTeam,teamId,championName,kills,deaths,...,n_gold,n_cs,n_champExperience,n_wardsPlaced,n_wardsKilled,n_heal,n_damageSelfMitigated,n_cc,n_damageTaken,rating


# Storing into a CSV file
Finally, we store the ratings into a CSV file. We will use this file to make the website to visualize the ratings.

In [256]:
destination_directory = "data_analysed/"

# Create the destination path by combining the destination directory and the original filename
destination_path = destination_directory + file_name

# Now store the ratings_dataset DataFrame in the destination path
ratings_dataset.to_csv(destination_path, index=False)

OSError: Cannot save file into a non-existent directory: 'data_analysed'